In [4]:
import pandas as pd

In [2]:
def multimer_write_pymol_scripts(delta_csv_path, pdb_dir_path, pymol_pLDDT_out_directory, pymol_pTM_out_directory, pymol_ipTM_out_directory,
                                 min_delta_pLDDT, max_delta_pLDDT, min_delta_pTM, max_delta_pTM, min_delta_ipTM, max_delta_ipTM):
    
    delta_df = pd.read_csv(delta_csv_path)

    full_sequences_df = delta_df[delta_df.which_seq == 0]
    
    for i, row in full_sequences_df.iterrows():
        complex_id = row.complex_id
        pdb_file_name = row.pdb_file
        pdb_file_path = os.path.join(pdb_dir_path, pdb_file_name)
        delta_mean_pLDDT_per_atom = []
        delta_pTM_per_atom = []
        delta_ipTM_per_atom = []

        with(open(pdb_file_path)) as f:

            cur_residue_num = '0'
            cur_delta_mean_pLDDT = 0
            cur_delta_pTM_per_atom = 0
            cur_delta_ipTM_per_atom = 0

            for line in f:

                split_line = line.split()
                indicator = split_line[0]

                # 'MODEL' is the indicator of the model... don't care about this line
                if indicator == 'MODEL':
                    continue

                elif indicator == 'ATOM':
                    
                    # If we are still on the same residue
                    if split_line[5] == cur_residue_num:
                        delta_mean_pLDDT_per_atom.append(cur_delta_mean_pLDDT)
                        delta_pTM_per_atom.append(cur_delta_pTM_per_atom)
                        delta_ipTM_per_atom.append(cur_delta_ipTM_per_atom)
                        continue

                    # If this is a new residue
                    else:
                        cur_residue_num = split_line[5]
                        cur_delta_mean_pLDDT = delta_df.loc[(delta_df.complex_id == complex_id) & (delta_df.delete_index == int(cur_residue_num) - 1)].delta_mean_pLDDT_score.to_numpy()[0]
                        cur_delta_pTM_per_atom = delta_df.loc[(delta_df.complex_id == complex_id) & (delta_df.delete_index == int(cur_residue_num) - 1)].delta_pTM_score.to_numpy()[0]
                        cur_delta_ipTM_per_atom = delta_df.loc[(delta_df.complex_id == complex_id) & (delta_df.delete_index == int(cur_residue_num) - 1)].delta_ipTM_score.to_numpy()[0]
                        delta_mean_pLDDT_per_atom.append(cur_delta_mean_pLDDT)
                        delta_pTM_per_atom.append(cur_delta_pTM_per_atom)
                        delta_ipTM_per_atom.append(cur_delta_ipTM_per_atom)
                        continue

                # TER indicates the last residue in the chain
                elif indicator == 'TER':

                    # If we are still on the same residue
                    # Note that TER has residue number as element 4
                    if split_line[4] == cur_residue_num:
                        delta_mean_pLDDT_per_atom.append(cur_delta_mean_pLDDT)
                        delta_pTM_per_atom.append(cur_delta_pTM_per_atom)
                        delta_ipTM_per_atom.append(cur_delta_ipTM_per_atom)
                        continue
                    
                    # If this is a new residue
                    else:
                        cur_residue_num = split_line[4]
                        cur_delta_mean_pLDDT = delta_df.loc[(delta_df.complex_id == complex_id) & (delta_df.delete_index == int(cur_residue_num) - 1)].delta_mean_pLDDT_score.to_numpy()[0]
                        cur_delta_pTM_per_atom = delta_df.loc[(delta_df.complex_id == complex_id) & (delta_df.delete_index == int(cur_residue_num) - 1)].delta_pTM_score.to_numpy()[0]
                        cur_delta_ipTM_per_atom = delta_df.loc[(delta_df.complex_id == complex_id) & (delta_df.delete_index == int(cur_residue_num) - 1)].delta_ipTM_score.to_numpy()[0]
                        delta_mean_pLDDT_per_atom.append(cur_delta_mean_pLDDT)
                        delta_pTM_per_atom.append(cur_delta_pTM_per_atom)
                        delta_ipTM_per_atom.append(cur_delta_ipTM_per_atom)
                        continue
                else:
                    continue

        # change extension of pdb to .pml
        pymol_file_path_pLDDT = os.path.join(pymol_pLDDT_out_directory, complex_id + '_delta_mean_pLDDT' + '.pml')
        pymol_file_path_pTM = os.path.join(pymol_pTM_out_directory, complex_id + '_delta_pTM' + '.pml')
        pymol_file_path_ipTM = os.path.join(pymol_ipTM_out_directory, complex_id + '_delta_ipTM' + '.pml')
        pymol_paths = [pymol_file_path_pLDDT, pymol_file_path_pTM, pymol_file_path_ipTM]
        delta_scores = [delta_mean_pLDDT_per_atom, delta_pTM_per_atom, delta_ipTM_per_atom ]
        min_delta_scores = [min_delta_pLDDT, min_delta_pTM, min_delta_ipTM]
        max_delta_scores = [max_delta_pLDDT, max_delta_pTM, max_delta_ipTM]

        score_names = ['delta_mean_pLDDT', 'delta_pTM', 'delta_ipTM']

        for pymol_path, delta_score, score_name, min, max in zip(pymol_paths, delta_scores, score_names, min_delta_scores, max_delta_scores):
            with open(pymol_path, "w") as f:
                f.write('load %s' % pdb_file_path + '\n')
                for i in range(1, len(delta_score) + 1):
                    f.write('set_atom_property' + ' ' + 'name=%s' % score_name + ', value=%s' % str(delta_score[i-1]) + ', selection=id %s' % i + ', proptype=3\n')

            with open(pymol_path, "a") as f:
                f.write('set_name %s' % (pdb_file_path.split(sep='/')[-1])[:-4] + ', %s' % complex_id + '_' + score_name + '\n')
                f.write('spectrum p.%s' % score_name + ', grey_red, %s' % complex_id + '_' + score_name + ', minimum=%s' % str(min) + ', maximum=%s' % str(max))

            #seq_to_pymol_file_dict[complex_id] = pymol_path

    return

In [3]:
def monomer_write_pymol_scripts(delta_csv_path, seq_id_df, pdb_dir_path, pymol_pLDDT_out_directory, pymol_pTM_out_directory,
                                min_delta_pLDDT, max_delta_pLDDT, min_delta_pTM, max_delta_pTM):
    
    delta_df = pd.read_csv(delta_csv_path)

    full_sequences_df = delta_df[delta_df.delete_index == -1]

    #seq_to_pymol_file_dict = {}
    
    for i, row in full_sequences_df.iterrows():
        sequence = row.seq
        complex_id = seq_id_df.loc[seq_id_df.seq == sequence].seq_id.to_numpy()[0]
        pdb_file_name = row.pdb_file
        pdb_file_path = os.path.join(pdb_dir_path, pdb_file_name)
        delta_mean_pLDDT_per_atom = []
        delta_pTM_per_atom = []

        with(open(pdb_file_path)) as f:

            cur_residue_num = '0'
            cur_delta_mean_pLDDT = 0
            cur_delta_pTM_per_atom = 0

            for line in f:

                split_line = line.split()
                indicator = split_line[0]

                # 'MODEL' is the indicator of the model... don't care about this line
                if indicator == 'MODEL':
                    continue

                elif indicator == 'ATOM':
                    
                    # If we are still on the same residue
                    if split_line[5] == cur_residue_num:
                        delta_mean_pLDDT_per_atom.append(cur_delta_mean_pLDDT)
                        delta_pTM_per_atom.append(cur_delta_pTM_per_atom)
                        continue

                    # If this is a new residue
                    else:
                        cur_residue_num = split_line[5]
                        cur_delta_mean_pLDDT = delta_df.loc[(delta_df.seq == sequence) & (delta_df.delete_index == int(cur_residue_num) - 1)].delta_mean_pLDDT_score.to_numpy()[0]
                        cur_delta_pTM_per_atom = delta_df.loc[(delta_df.seq == sequence) & (delta_df.delete_index == int(cur_residue_num) - 1)].delta_pTM_score.to_numpy()[0]
                        delta_mean_pLDDT_per_atom.append(cur_delta_mean_pLDDT)
                        delta_pTM_per_atom.append(cur_delta_pTM_per_atom)
                        continue

                # TER indicates the last residue in the chain
                elif indicator == 'TER':

                    # If we are still on the same residue
                    # Note that TER has residue number as element 4
                    if split_line[4] == cur_residue_num:
                        delta_mean_pLDDT_per_atom.append(cur_delta_mean_pLDDT)
                        delta_pTM_per_atom.append(cur_delta_pTM_per_atom)
                        continue
                    
                    # If this is a new residue
                    else:
                        cur_residue_num = split_line[4]
                        cur_delta_mean_pLDDT = delta_df.loc[(delta_df.seq == sequence) & (delta_df.delete_index == int(cur_residue_num) - 1)].delta_mean_pLDDT_score.to_numpy()[0]
                        cur_delta_pTM_per_atom = delta_df.loc[(delta_df.seq == sequence) & (delta_df.delete_index == int(cur_residue_num) - 1)].delta_pTM_score.to_numpy()[0]
                        delta_mean_pLDDT_per_atom.append(cur_delta_mean_pLDDT)
                        delta_pTM_per_atom.append(cur_delta_pTM_per_atom)
                        continue
                else:
                    continue

        # change extension of pdb to .pml
        pymol_file_path_pLDDT = os.path.join(pymol_pLDDT_out_directory, complex_id + '_delta_mean_pLDDT' + '.pml')
        pymol_file_path_pTM = os.path.join(pymol_pTM_out_directory, complex_id + '_delta_pTM' + '.pml')
        pymol_paths = [pymol_file_path_pLDDT, pymol_file_path_pTM]
        delta_scores = [delta_mean_pLDDT_per_atom, delta_pTM_per_atom]
        min_delta_scores = [min_delta_pLDDT, min_delta_pTM]
        max_delta_scores = [max_delta_pLDDT, max_delta_pTM]

        score_names = ['delta_mean_pLDDT', 'delta_pTM']

        for pymol_path, delta_score, score_name, min, max in zip(pymol_paths, delta_scores, score_names, min_delta_scores, max_delta_scores):
            with open(pymol_path, "w") as f:
                f.write('load %s' % pdb_file_path + '\n')
                for i in range(1, len(delta_score) + 1):
                    f.write('set_atom_property' + ' ' + 'name=%s' % score_name + ', value=%s' % str(delta_score[i-1]) + ', selection=id %s' % i + ', proptype=3\n')

            with open(pymol_path, "a") as f:
                f.write('set_name %s' % (pdb_file_path.split(sep='/')[-1])[:-4] + ', %s' % complex_id + '_' + score_name + '\n')
                f.write('spectrum p.%s' % score_name + ', grey_red, %s' % complex_id + '_' + score_name + ', minimum=%s' % str(min) + ', maximum=%s' % str(max))
            #seq_to_pymol_file_dict[sequence] = pymol_path

    return

In [ ]:
# Multimer write to pymol

delta_csv_path = '../processed_deletion_perturb_out/processed_multimer_output.csv'
pdb_dir_path = '../selected_pdb_files/multimer_pdb'
pymol_pLDDT_out_directory = '../pymol/multimer_pymol_scripts/pLDDT'
pymol_pTM_out_directory = '../pymol/multimer_pymol_scripts/pTM'
pymol_ipTM_out_directory = '../pymol/multimer_pymol_scripts/ipTM'


multimer_write_pymol_scripts(delta_csv_path, pdb_dir_path, pymol_pLDDT_out_directory, pymol_pTM_out_directory, pymol_ipTM_out_directory,
                             0, 100, 0, 1, 0, 1)

In [ ]:
# Monomer write to pymol

# First, get the sequence ID from the multimer csv and match with monomer csv

processed_multimer_df = pd.read_csv('../processed_deletion_perturb_out/processed_multimer_output.csv')

seq_id_df = processed_multimer_df[processed_multimer_df.which_seq == 0][['complex_id', 'seq1', 'seq2']]

seq_id_df[['seq1_id','seq2_id']] = seq_id_df.complex_id.str.split('A_', expand=True)
seq_id_df['seq1_id'] = seq_id_df['seq1_id'].astype('str') + 'A'
seq_id_df = seq_id_df.drop('complex_id', axis=1)
stack1 = seq_id_df[['seq1','seq1_id']]
stack2 = seq_id_df[['seq2','seq2_id']].rename(columns={'seq2':'seq1','seq2_id':'seq1_id'})
seq_id_df = pd.concat((stack1, stack2)).drop_duplicates(subset='seq1').rename(columns={'seq1':'seq', 'seq1_id':'seq_id'}) # probably don't even need drop duplicates

# Next write the pymol scripts

delta_csv_path = '../processed_deletion_perturb_out/processed_monomer_output.csv'
pdb_dir_path = '../selected_pdb_files/monomer_pdb'
pymol_pLDDT_out_directory = '../pymol/monomer_pymol_scripts/pLDDT'
pymol_pTM_out_directory = '../pymol/monomer_pymol_scripts/pTM'

monomer_write_pymol_scripts(delta_csv_path, seq_id_df, pdb_dir_path, pymol_pLDDT_out_directory, pymol_pTM_out_directory,
                            0, 100, 0, 1)

In [ ]:
# Antigen antibody multimer preparation for pymol
AA_multimer_delta_csv_path = '../processed_deletion_perturb_out/processed_AA_multimer_output.csv'
AA_multimer_pdb_dir_path = '../selected_pdb_files/AA_multimer_pdb'
AA_pymol_multimer_pLDDT_out_directory = '../pymol/AA_multimer_pymol_scripts/pLDDT'
AA_pymol_multimer_pTM_out_directory = '../pymol/AA_multimer_pymol_scripts/pTM'
AA_pymol_multimer_ipTM_out_directory = '../pymol/AA_multimer_pymol_scripts/ipTM'

multimer_write_pymol_scripts(AA_multimer_delta_csv_path,
                             AA_multimer_pdb_dir_path,
                             AA_pymol_multimer_pLDDT_out_directory,
                             AA_pymol_multimer_pTM_out_directory,
                             AA_pymol_multimer_ipTM_out_directory,
                             0, 10, 0, 0.1, 0, 0.1)

In [ ]:
# Antigen-antibody monomer preparation for pymol

# First the sequence ID from the AA multimer csv and match with AA monomer csv

processed_AA_multimer_df = pd.read_csv('../processed_deletion_perturb_out/processed_AA_multimer_output.csv')

AA_seq_id_df = processed_AA_multimer_df[processed_AA_multimer_df.which_seq == 0][['complex_id', 'seq1', 'seq2']]

AA_seq_id_df[['seq1_id','seq2_id']] = AA_seq_id_df.complex_id.str.split('A_', expand=True)
AA_seq_id_df['seq1_id'] = AA_seq_id_df['seq1_id'].astype('str') + 'A'
AA_seq_id_df = AA_seq_id_df.drop('complex_id', axis=1)
AA_stack1 = AA_seq_id_df[['seq1','seq1_id']]
AA_stack2 = AA_seq_id_df[['seq2','seq2_id']].rename(columns={'seq2':'seq1','seq2_id':'seq1_id'})
AA_seq_id_df = pd.concat((AA_stack1, AA_stack2)).drop_duplicates(subset='seq1').rename(columns={'seq1':'seq', 'seq1_id':'seq_id'}) # probably don't even need drop duplicates


# Next write the pymol scripts
AA_monomer_delta_csv_path = '../processed_deletion_perturb_out/processed_AA_monomer_output.csv'
AA_monomer_pdb_dir_path = '../selected_pdb_files/AA_monomer_pdb'
# pymol_pLDDT_out_directory and pymol_pTM_out_directory SHOULD (maybe MUST) be absolute file paths
AA_pymol_monomer_pLDDT_out_directory = '../pymol/AA_monomer_pymol_scripts/pLDDT'
AA_pymol_monomer_pTM_out_directory = '../pymol/AA_monomer_pymol_scripts/pTM'

monomer_write_pymol_scripts(AA_monomer_delta_csv_path,
                            AA_seq_id_df,
                            AA_monomer_pdb_dir_path,
                            AA_pymol_monomer_pLDDT_out_directory,
                            AA_pymol_monomer_pTM_out_directory,
                            0, 10, 0, 0.1)